In [3]:
import func2
from modulos import *
import pandas as pd
aba_aberta = False
nami = Firefox()
d = func2.Logar(nami)

df = pd.read_excel('arq.xlsx')
del df[df.columns[0]] # serve para deletar a coluna index

nami.get('https://www.binance.com/pt-BR/markets/spot-FIAT/BRL')
precos = nami.find_elements_by_css_selector('div[class = "css-1iif8xd"]')
links_html = nami.find_elements_by_css_selector('a[class = "css-1sud65k"]')
pega_links = lambda x: x.get_attribute('href')
pega_preco = lambda x: float(x.text.replace('.', '').replace(',', '.'))
links = list(map(pega_links, links_html))
new_price = list(map(pega_preco, precos))
df['new_price'] = new_price
#df['lins2'] = links
#df.to_excel('arq3.xlsx')

ValueError: Length of values (31) does not match length of index (128)

In [4]:
# Calculo da valorizacao da moeda
var_moeda = lambda xf, xi: (xf - xi)/xi
v_moeda = list(map(var_moeda, df.new_price, df.buy_price))

AttributeError: 'DataFrame' object has no attribute 'buy_price'

In [40]:

def calc_lucro(chz_i, x2, x1):
    return chz_i*(x2-x1)

def tranferencia(lucro):
    return lucro/(2-TX)

def transferindo(t_1, chz, brl, x2):
    #vende
    if t_1 > 0: 
        brl_t = brl + t_1*(1-TX)
        chz_t = chz - t_1/x2
    #compra
    elif t_1 < 0: 
        brl_t = brl + t_1
        chz_t = chz - (t_1/x2)*(1-TX)
    else:
        brl_t = brl
        chz_t = chz
    return brl_t, chz_t

def calc_ganho_real(brl_f, chz_f, chz_i, brl_i):    
    brl_g = brl_f - brl_i
    chz_g = chz_f - chz_i
    p_brl_g = ((brl_f - brl_i)/brl_i)*100
    p_chz_g = ((chz_f - chz_i)/chz_i)*100
    return brl_g, chz_g, p_brl_g, p_chz_g


TX = 0.001
# indo
l_1 = list(map(calc_lucro, df.chz_i, df.new_price, df.buy_price))
t_1 = list(map(tranferencia, l_1))
brl_t, chz_t = zip(*map(transferindo, t_1, df.chz_i, df.brl_i, df.new_price))

#vindo
l_2 = list(map(calc_lucro, chz_t, df.buy_price, df.new_price))
t_2 = list(map(tranferencia, l_2))
brl_f, chz_f = zip(*map(transferindo, t_2, chz_t, brl_t, df.buy_price))

#Calc do ganho real
brl_g, chz_g, p_brl_g, p_chz_g = zip(*map(calc_ganho_real, brl_f, chz_f, df.chz_i, df.brl_i))
df['p_brl_g'] = p_brl_g
df['p_chz_g'] = p_chz_g
df.to_excel('arq3.xlsx')

In [42]:
links_transf = []
cond_g_min = 5
cond_v_moeda = 5
for x in range(len(df)):
    if abs(v_moeda[x]) > cond_v_moeda:
        if abs(p_brl_g[x]) >= cond_g_min:
            links_transf.append(x)
links_transf

[]

In [31]:
def ord_compra(brl, nami):
    sleep(5)
    nami.find_element_by_css_selector('input[id = "FormRow-BUY-total"]').clear()
    nami.find_element_by_css_selector('input[id = "FormRow-BUY-total"]').send_keys(str(brl))

def ord_venda(brl, nami):
    sleep(10)
    nami.find_element_by_css_selector('input[id = "FormRow-SELL-total"]').clear()
    nami.find_element_by_css_selector('input[id = "FormRow-SELL-total"]').send_keys(str(abs(brl)))

limite_order = 10

for x in links_transf:    
    df.buy_price[x] = new_price[x]
    if t_1[x] > limite_order:
        nami.get(df.links[x])
        ord_compra(t_1[x], nami)
    elif t_1[x] < -limite_order:
        nami.get(df.link[x])
        ord_venda(t_1[x], nami)



df.to_excel('novo.xlsx')

[]


In [ ]:

l_1, l_2 = [], []
t_1, t_2 = [], []
brl_t, brl_f, brl_g, chz_t, chz_f, chz_g = [], [], [], [], [], []
porc_brl, porc_chz = [], []
for x in range(len(df)):
    x1= df.buy_price[x]
    x2 = df.new_price[x]
    chz_i = df.chz_i[x]
    brl_i = df.brl_i[x]
    l_1.append(calc_lucro(chz_i, x2, x1))
    t_1.append(tranferencia(l_1[x], TX))

    brl, chz = tranferindo(t_1[x], TX, chz_i, brl_i, x2)
    brl_t.append(brl)
    chz_t.append(chz)

    l_2.append(calc_lucro(chz_t[x], x1, x2))
    t_2.append(tranferencia(l_2[x], TX))

    brl2, chz2 = tranferindo(t_2[x], TX, chz_t[x], brl_t[x], x1)
    brl_f.append(brl2)
    chz_f.append(chz2)

    brl_gg, chz_gg, p_brl_gg, p_chz_gg = calc_ganho_real(brl_f[x], chz_f[x], chz_i, brl_i)
    brl_g.append(brl_gg)
    chz_g.append(chz_gg)
    porc_brl.append(p_brl_gg)
    porc_chz.append(p_chz_gg)